###Encoding Text
As we know, machine learning models and neural networks don't take raw text data as an input. This means we must somehow encode our textual data to numeric values that our models can understand. There are many different ways of doing this, and we will look at a few examples below.
Before we get into the different encoding/preprocessing methods, let's understand the information we can get from textual data by looking at the following two movie reviews:
"I thought the movie was going to be bad, but it was actually amazing!"
"I thought the movie was going to be amazing, but it was actually bad!"
Although these two sentences are very similar, we know that they have very different meanings. This is because of the ordering of words, a very important property of textual data. Now keep that in mind while we consider some different ways of encoding our textual data.

###Bag of Words
The first and simplest way to encode our data is to use something called bag of words. This is a pretty easy technique where each word in a sentence is encoded with an integer and thrown into a collection that does not maintain the order of the words but does keep track of the frequency. Have a look at the Python function below that encodes a string of text into bag of words.

In [1]:
vocab = {}  # maps word to integer representing it
word_encoding = 1
def bag_of_words(text):
  global word_encoding

  words = text.lower().split(" ")  # create a list of all of the words in the text, well assume there is no grammar in our text for this example
  bag = {}  # stores all of the encodings and their frequency

  for word in words:
    if word in vocab:
      encoding = vocab[word]  # get encoding from vocab
    else:
      vocab[word] = word_encoding
      encoding = word_encoding
      word_encoding += 1

    if encoding in bag:
      bag[encoding] += 1
    else:
      bag[encoding] = 1

  return bag

text = "this is a test to see if this test will work is is test a a"
bag = bag_of_words(text)
print(bag)
print(vocab)

{1: 2, 2: 3, 3: 3, 4: 3, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1}
{'this': 1, 'is': 2, 'a': 3, 'test': 4, 'to': 5, 'see': 6, 'if': 7, 'will': 8, 'work': 9}


In [2]:
positive_review = "I thought the movie was going to be bad but it was actually amazing"
negative_review = "I thought the movie was going to be amazing but it was actually bad"

pos_bag = bag_of_words(positive_review)
neg_bag = bag_of_words(negative_review)

print("Positive:", pos_bag)
print("Negative:", neg_bag)

Positive: {10: 1, 11: 1, 12: 1, 13: 1, 14: 2, 15: 1, 5: 1, 16: 1, 17: 1, 18: 1, 19: 1, 20: 1, 21: 1}
Negative: {10: 1, 11: 1, 12: 1, 13: 1, 14: 2, 15: 1, 5: 1, 16: 1, 21: 1, 18: 1, 19: 1, 20: 1, 17: 1}


We can see that even though these sentences have a very different meaning they are encoded exaclty the same way. Obviously, this isn't going to fly. Let's look at some other methods.



###Integer Encoding
The next technique we will look at is called integer encoding. This involves representing each word or character in a sentence as a unique integer and maintaining the order of these words. This should hopefully fix the problem we saw before were we lost the order of words.
To do this, we first need to create a mapping between each unique word and an integer. We can do this by creating a dictionary that maps each word to a unique integer. Then, when we want to encode a sentence, we simply replace each word with its corresponding integer.

In [3]:
vocab = {}
word_encoding = 1
def one_hot_encoding(text):
  global word_encoding

  words = text.lower().split(" ")
  encoding = []

  for word in words:
    if word in vocab:
      code = vocab[word]
      encoding.append(code)
    else:
      vocab[word] = word_encoding
      encoding.append(word_encoding)
      word_encoding += 1

  return encoding

text = "this is a test to see if this test will work is is test a a"
encoding = one_hot_encoding(text)
print(encoding)
print(vocab)

[1, 2, 3, 4, 5, 6, 7, 1, 4, 8, 9, 2, 2, 4, 3, 3]
{'this': 1, 'is': 2, 'a': 3, 'test': 4, 'to': 5, 'see': 6, 'if': 7, 'will': 8, 'work': 9}


In [4]:
positive_review = "I thought the movie was going to be bad but it was actually amazing"
negative_review = "I thought the movie was going to be amazing but it was actually bad"

pos_encode = one_hot_encoding(positive_review)
neg_encode = one_hot_encoding(negative_review)

print("Positive:", pos_encode)
print("Negative:", neg_encode)

Positive: [10, 11, 12, 13, 14, 15, 5, 16, 17, 18, 19, 14, 20, 21]
Negative: [10, 11, 12, 13, 14, 15, 5, 16, 21, 18, 19, 14, 20, 17]


Much better, now we are keeping track of the order of words and we can tell where each occurs. But this still has a few issues with it. Ideally, when we encode words, we would like similar words to have similar labels and different words to have very different labels. For example, the words "happy" and "joyful" should probably have very similar labels so we can determine that they are similar. While words like "horrible" and "amazing" should probably have very different labels. The method we looked at above won't be able to do something like this for us. This could mean that the model will have a very difficult time determining if two words are similar or not, which could result in some pretty drastic performance impacts.

###Word Embeddings
Luckily, there is a third method that is far superior, word embeddings. This method keeps the order of words intact as well as encodes similar words with very similar labels. It attempts to not only encode the frequency and order of words but the meaning of those words in the sentence. It encodes each word as a dense vector that represents its context in the sentence.
Unlike the previous techniques, word embeddings are learned by looking at many different training examples. You can add what's called an embedding layer to the beginning of your model, and while your model trains, your embedding layer will learn the correct embeddings for words. You can also use pre-trained embedding layers.

In [5]:
import numpy as np

def word_encoding(text):
    # Define a simple word embedding matrix
    word_embeddings = {
        "I": np.array([1, 0, 0]),
        "thought": np.array([0, 1, 0]),
        "the": np.array([0, 0, 1]),
        "movie": np.array([1, 1, 0]),
        "was": np.array([0, 1, 1]),
        "going": np.array([1, 0, 1]),
        "to": np.array([1, 1, 1]),
        "be": np.array([0, 0, 0]),
        "bad": np.array([-1, 0, 0]),
        "but": np.array([0, -1, 0]),
        "it": np.array([0, 0, -1]),
        "actually": np.array([0, 0, 0]),
        "amazing": np.array([1, 1, 1])
    }

    encoded_text = []
    for word in text.lower().split():
        if word in word_embeddings:
            encoded_text.append(word_embeddings[word])
        else:
            encoded_text.append(np.zeros(3))  # Handle unknown words

    return encoded_text

In [6]:
positive_review = "I thought the movie was going to be bad but it was actually amazing"
negative_review = "I thought the movie was going to be amazing but it was actually bad"

pos_encode = word_encoding(positive_review)
neg_encode = word_encoding(negative_review)

print("Positive:", pos_encode)
print("Negative:", neg_encode)

Positive: [array([0., 0., 0.]), array([0, 1, 0]), array([0, 0, 1]), array([1, 1, 0]), array([0, 1, 1]), array([1, 0, 1]), array([1, 1, 1]), array([0, 0, 0]), array([-1,  0,  0]), array([ 0, -1,  0]), array([ 0,  0, -1]), array([0, 1, 1]), array([0, 0, 0]), array([1, 1, 1])]
Negative: [array([0., 0., 0.]), array([0, 1, 0]), array([0, 0, 1]), array([1, 1, 0]), array([0, 1, 1]), array([1, 0, 1]), array([1, 1, 1]), array([0, 0, 0]), array([1, 1, 1]), array([ 0, -1,  0]), array([ 0,  0, -1]), array([0, 1, 1]), array([0, 0, 0]), array([-1,  0,  0])]
